In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"

/*import libtorch header file*/
#include <torch/torch.h>

# 线性回归的简单实现：使用libtorch API

In [2]:
typedef std::tuple<torch::Tensor, torch::Tensor> sample_set;

In [3]:
sample_set synthetic_data(torch::Tensor w, torch::Tensor b, int num_example)
{
    torch::Tensor X = torch::normal(0.0, 1.0, {num_example, w.size(0)}); 
    torch::Tensor y = X.matmul(w) + b;
    y = y + torch::normal(0.0, 0.1, y.sizes());
    
    return std::make_tuple(X, y);
}

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
    
    def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

In [ ]:
float coe[] = {2.0, 3.4};
float bias[] = {4.2};
int num_example = 20;
int batch_size = 4;
/** calling 'from_blob(...)' may cause compile error with my
 *  xeus-clilng enviroment, so here I changed it with another
 *  way; but maybe you can try it in your pc;
 */
// torch::Tensor true_w = torch::from_blob(coe, {2, 1});
// torch::Tensor true_b = torch::from_blob(bias, {1});

torch::Tensor true_w = torch::tensor(at::ArrayRef<float>({2.0, -3.4}));
torch::Tensor true_b = torch::tensor(at::ArrayRef<float>({4.2}));
// sample_set data = synthetic_data(true_w, true_b, 1000);

torch::Tensor X = torch::normal(0.0, 1.0, {num_example, w.size(0)}); 
torch::Tensor y = X.matmul(w) + b;
y = y + torch::normal(0.0, 0.1, y.sizes());

std::vector<torch::Tensor> tlist;
tlist.push_back(X);
tlist.push_back(y);

auto dataset = torch::data::datasets::TensorDataset(tlist);
auto data_loader = torch::data::make_data_loader<torch::data::samplers::RandomSampler>(
    std::move(dataset), 
    batch_size);
for (auto& batch : *data_loader) {
    auto data = batch.data;
    auto labels = batch.target;
    // do your usual stuff
    printT(data);
}

input_line_21:15:57: error: use of undeclared identifier 'w'
torch::Tensor X = torch::normal(0.0, 1.0, {num_example, w.size(0)}); 
                                                        ^
input_line_21:16:28: error: use of undeclared identifier 'w'
torch::Tensor y = X.matmul(w) + b;
                           ^
input_line_21:16:33: error: use of undeclared identifier 'b'
torch::Tensor y = X.matmul(w) + b;
                                ^
input_line_21:26:23: error: reference to non-static member function must be called; did you mean
      to call it with no arguments?
    auto data = batch.data;
                ~~~~~~^~~~
                          ()
/Users/liuhang/anaconda3/envs/mycpp/include/c++/v1/vector:696:23: note: possible target for call
    value_type*       data() _NOEXCEPT
                      ^
/Users/liuhang/anaconda3/envs/mycpp/include/c++/v1/vector:699:23: note: possible target for call
    const value_type* data() const _NOEXCEPT
                      ^
input_line_21

In [ ]:
xxx load_array(data_arrays, batch_size, is_train=True)
{
    torch::data::datasets::TensorDataset dataset = torch::data::datasets::TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)    
}